<a href="https://colab.research.google.com/github/avinashmane/ainvest/blob/master/notebooks/american_option_pricing_binomial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
%pip install pydash
%pip install yfinance
import math
from rich.pretty import pprint
import ipywidgets as w
from IPython.display import display,Markdown as md
from rich.pretty import pprint
from pydash import pick
import pandas as pd
import numpy as np


In [210]:
pd.set_option("display.precision", 2)
RATE=0.04
N=100

def american_option_pricing(S0, K, T, sigma, r=RATE, N=N, option_type='Put'):
    """
    Prices an American Call or Put option using the Binomial Option Pricing Model (BOPM).

    This model is necessary for American options because it allows for the early
    exercise decision to be checked at every node (time step) in the lattice.

    Args:
        S0 (float): Current Stock Price.
        K (float): Strike Price.
        T (float): Time to Expiration (in years).
        r (float): Risk-Free Interest Rate (annualized, continuous compounding).
        sigma (float): Volatility of the underlying asset (annualized).
        N (int): Number of time steps/periods in the lattice.
        option_type (str): 'Call' or 'Put'. Defaults to 'Put' (the option
                           most likely to be exercised early).

    Returns:
        float: The calculated American option price.
    """

    # --- 1. Calculate Time Step and Binomial Parameters ---

    # Time step duration
    dt = T / N

    # Up factor (u) and Down factor (d) - Cox-Ross-Rubinstein (CRR) method
    # This structure ensures the tree is recombining and arbitrage-free.
    u = math.exp(sigma * math.sqrt(dt))
    d = 1 / u

    # Risk-neutral probability (p)
    # The expected return is the risk-free rate (e^(r*dt)).
    a = math.exp(r * dt)
    p = (a - d) / (u - d)

    # Discount factor
    discount_factor = math.exp(-r * dt)

    # --- 2. Initialize Stock Prices at Expiration (Time T) ---

    # The tree has N+1 terminal nodes.
    # The j-th node (0 to N) represents j 'up' movements and N-j 'down' movements.
    stock_prices = [0.0] * (N + 1)

    # Calculate stock price at each terminal node j
    for j in range(N + 1):
        stock_prices[j] = S0 * (u ** j) * (d ** (N - j))

    # print(",".join(map(lambda x:f"{x:0.2f}",stock_prices)))
    # --- 3. Calculate Option Payoffs at Expiration (Time T) ---

    # Initialize the option value list using the final payoffs
    option_values = [0.0] * (N + 1)

    if option_type == 'Call':
        for j in range(N + 1):
            # Payoff is max(S(T) - K, 0)
            option_values[j] = max(0, stock_prices[j] - K)
    elif option_type == 'Put':
        for j in range(N + 1):
            # Payoff is max(K - S(T), 0)
            option_values[j] = max(0, K - stock_prices[j])
    else:
        raise ValueError("option_type must be 'Call' or 'Put'")

    # --- 4. Work Backwards (Backward Induction) ---

    # Loop backwards from time step N-1 down to 0
    for i in range(N - 1, -1, -1):
        # We need to calculate i+1 option values for the current time step i
        for j in range(i + 1):

            # a. Calculate Continuation Value (European-style)
            # This is the discounted expected value from the next time step (i+1)
            continuation_value = discount_factor * (
                p * option_values[j+1] +        # Value if stock moves Up
                (1 - p) * option_values[j]      # Value if stock moves Down
            )

            # b. Calculate Intrinsic Value (Immediate Exercise)
            # Find the stock price at the current node (i, j)
            Si = S0 * (u ** j) * (d ** (i - j))

            if option_type == 'Call':
                intrinsic_value = max(0, Si - K)
            else: # Put
                intrinsic_value = max(0, K - Si)

            # c. American Option Valuation (Early Exercise Check)
            # The American option value is the maximum of holding (continuation) or exercising now (intrinsic).
            option_values[j] = max(intrinsic_value, continuation_value)

            # Note: The option_values array is overwritten in place, reducing
            # the size by one at each step back, keeping only the necessary
            # values for the next step.

    # --- 5. Return the Result ---
    # The final value remaining at index 0 is the option price at time t=0.
    return option_values[0]

# --- Example Usage ---

# Parameters for an American Put
S0 = 50.0       # Current stock price
K = 50.0        # Strike price
T = 0.5         # 6 months to expiration
r = 0.05        # 5% risk-free rate
sigma = 0.30    # 30% volatility
N = 100         # Number of steps (higher N = more accuracy, slower calculation)

# Price the American Put
put_price = american_option_pricing(
    S0=S0,
    K=K,
    T=T,
    sigma=sigma,
    r=r,
    N=N,
    option_type='Put'
)

print(f"--- American Option Pricing (Binomial Model) ---")
print(f"Stock Price (S0): ${S0}")
print(f"Strike Price (K): ${K}")
print(f"Time to Expiration (T): {T} years")
print(f"Risk-Free Rate (r): {r*100}%")
print(f"Volatility (sigma): {sigma*100}%")
print(f"Number of Steps (N): {N}\n")
print(f"Calculated American Put Price: ${put_price:.4f}")

# Example for an American Call (Note: American Calls are rarely worth exercising early unless there are dividends)
call_price = american_option_pricing(
    S0=S0,
    K=K,
    T=T,
    sigma=sigma,
    r=r,
    N=N,
    option_type='Call'
)
print(f"Calculated American Call Price: ${call_price:.4f}")

--- American Option Pricing (Binomial Model) ---
Stock Price (S0): $50.0
Strike Price (K): $50.0
Time to Expiration (T): 0.5 years
Risk-Free Rate (r): 5.0%
Volatility (sigma): 30.0%
Number of Steps (N): 100

Calculated American Put Price: $3.6913
Calculated American Call Price: $4.8070


In [211]:
import yfinance as yf
class MyTicker(yf.Ticker):
  stock_data={}
  @property
  def price(self)  -> float:
    return self.info['currentPrice']
  def download(self,
         period : str ="1y") :
      if not period in self.stock_data:
        self.stock_data[period] = yf.download(ticker.info['symbol'], period=period, auto_adjust = True)
      return self.stock_data[period]

  @property
  def hv(self,
         period : str ="1y") -> float:
    try:
      stock_data = self.download(period)
      # Calculate daily returns
      stock_data['Daily Return'] = stock_data['Close'].pct_change()
      # Calculate daily volatility (standard deviation of daily returns)
      daily_volatility = stock_data['Daily Return'].std()
      # Annualize volatility (assuming 252 trading days in a year)
      annualized_volatility = daily_volatility * np.sqrt(252)
      return float(annualized_volatility)
    except Exception as e:
      print(f"Error calculating historical volatility for {ticker_symbol}: {e!r}")
      return None

In [212]:
ticker=MyTicker('ADBE')

In [213]:

ticker=MyTicker("ADBE")
# ticker.options[4:9]

def opt_quotes(price,quotes=20,step_size=10):
  start=int(price/step_size)*step_size-quotes/2*step_size
  # print(start)
  return [start+x*step_size for x in range(quotes)]

def get_price_array(price,  expiry, rate, hv, step_size=10, quotes=16):

  options=[]

  if type(expiry)==str:
    expiry=[expiry]

  for strike in opt_quotes(price,quotes,step_size):
    for _expiry in expiry:
      # print(price, strike, option_DTE(_expiry), hv)
      opt=dict(
          strike=strike,
          expiry=_expiry,
          call=american_option_pricing(price, strike, option_DTE(_expiry), hv, rate, N, option_type='Call'),
          put=american_option_pricing(price, strike, option_DTE(_expiry), hv, rate, N, option_type='Put'))
      options.append(opt)
  # print(i,*opt.values())
  return options

# Define N here, as it is used in get_price_array
N = 1000

# Define option_DTE function, as it is used in get_price_array
from datetime import datetime
def option_DTE(expiry_date_str):
    """Calculates Days to Expiration (DTE) from an expiry date string."""
    expiry_date = datetime.strptime(expiry_date_str, '%Y-%m-%d')
    today = datetime.now()
    delta = expiry_date - today
    return delta.days / 365.25  # Convert days to years


options=  get_price_array(ticker.price,  '2025-11-21', .04, .44,)
df=pd.DataFrame(options)
# df

333.26 250.0 0.08487337440109514 0.44
333.26 260.0 0.08487337440109514 0.44
333.26 270.0 0.08487337440109514 0.44
333.26 280.0 0.08487337440109514 0.44
333.26 290.0 0.08487337440109514 0.44
333.26 300.0 0.08487337440109514 0.44
333.26 310.0 0.08487337440109514 0.44
333.26 320.0 0.08487337440109514 0.44
333.26 330.0 0.08487337440109514 0.44
333.26 340.0 0.08487337440109514 0.44
333.26 350.0 0.08487337440109514 0.44
333.26 360.0 0.08487337440109514 0.44
333.26 370.0 0.08487337440109514 0.44
333.26 380.0 0.08487337440109514 0.44
333.26 390.0 0.08487337440109514 0.44
333.26 400.0 0.08487337440109514 0.44


In [217]:
"Option Price for next 4:9 expiries for n quotes"
quotes=16
@w.interact_manual
def get_ticker(t="ADBE"):
  global df,ticker
  ticker = MyTicker(t)
  price=ticker.price

  expiry=ticker.options[4:9]#['2025-11-21']
  rate=.04
  hv=ticker.hv
  N=1000
  print(pick(ticker.info,"shortName"),expiry)

  options=  get_price_array(price, expiry, hv, rate, quotes=quotes)
  df=pd.DataFrame(options)
  display(alt.Chart(df).mark_line(interpolate='basis',point=True).encode(
    x="strike",
    y="call",
    color="expiry",
    tooltip=list(df.columns)).interactive().properties(width=800,height=600))
  return "{}: {}".format(t,ticker.info['shortName'])
# ticker.options


interactive(children=(Text(value='ADBE', description='t'), Button(description='Run Interact', style=ButtonStyl…

In [198]:

df_2=df.copy()
base=alt.Chart(df_2).mark_line(interpolate='basis',point=True).interactive().properties(width=800,height=600)
chart_price=base.encode(
    x="strike",
    y="call",
    color="expiry",
    tooltip=list(df.columns))


def add_gain(df,
            strike=280,
            expiry="2025-11-21"):
  df_3=df.query('expiry==@expiry').copy()
  match_x=df_3.query('strike==@strike and expiry==@expiry')
  cost=match_x['call'].iloc[0]
  df_3['cost']= cost
  return (df_3['strike'].apply(lambda x : max(0,x-strike)) - cost ) / cost

for _strike in [280,290,300,310,320]:
  for _expiry in ['2025-11-21']:
    df_3[f'gain_{_expiry}_{_strike}']=add_gain(df_2, _strike, _expiry)

df_3['gain_stock']=(df_3['strike']-ticker.price)/ticker.price
chart_gain=alt.Chart(df_3).encode(
    x="strike",
    y="gain_stock",
    color="expiry",
    tooltip=list(df.columns))
# alt.layer(chart_price, chart_gain).resolve_scale(
#     y='independent'
# )
df_3

,strike,expiry,call,put,cost,gain_280_2025-11-21,gain_stock,gain_2025-11-21_280,gain_2025-11-21_290,gain_2025-11-21_300,gain_2025-11-21_310,gain_2025-11-21_320
0,230.0,2025-11-21,1.12e+02,0.00e+00,63.52,-1.00,-3.10e-01,-1.0,-1.0,-1.0,-1.0,-1.0
1,240.0,2025-11-21,1.02e+02,2.95e-254,63.52,-1.00,-2.80e-01,NaN,NaN,NaN,NaN,NaN
2,250.0,2025-11-21,9.24e+01,6.49e-191,63.52,-1.00,-2.50e-01,NaN,NaN,NaN,NaN,NaN
3,260.0,2025-11-21,8.28e+01,3.84e-143,63.52,-1.00,-2.20e-01,NaN,NaN,NaN,NaN,NaN
4,270.0,2025-11-21,7.32e+01,5.55e-106,63.52,-1.00,-1.90e-01,NaN,NaN,NaN,NaN,NaN
5,280.0,2025-11-21,6.35e+01,1.18e-76,63.52,-1.00,-1.60e-01,-1.0,-1.0,-1.0,-1.0,-1.0
6,290.0,2025-11-21,5.39e+01,1.85e-53,63.52,-0.84,-1.30e-01,NaN,NaN,NaN,NaN,NaN
7,300.0,2025-11-21,4.43e+01,2.10e-35,63.52,-0.69,-9.98e-02,NaN,NaN,NaN,NaN,NaN
8,310.0,2025-11-21,3.46e+01,1.14e-21,63.52,-0.53,-6.98e-02,NaN,NaN,NaN,NaN,NaN
9,320.0,2025-11-21,2.50e+01,1.59e-11,63.52,-0.37,-3.98e-02,NaN,NaN,NaN,NaN,NaN


In [195]:
import altair as alt
# df=pd.DataFrame(options)
_cols=['strike','f','v']
alt.Chart(df_3.set_index('strike')[[x for x in df_3.columns if 'gain' in x]].stack().reset_index().set_axis(_cols,axis=1)
  ).mark_line(interpolate='basis',point=True).encode(
    x="strike",
    y="v",
    color="f",
    tooltip=_cols).interactive().properties(width=800,height=600)

alt.Chart(...)

In [191]:

cols="contractSymbol strike 	lastPrice 	bid 	ask 	change 	volume 	openInterest 	impliedVolatility 	inTheMoney".split()
df_calls=ticker.option_chain().calls[cols].set_index('contractSymbol')

opt_price={}
strike=280
expiry='2025-11-21'
opt_price[strike]=american_option_pricing(ticker.price, strike, option_DTE(expiry), ticker.hv, option_type='Call')
df_calls[f'gain_{strike}']=df_calls['strike']/opt_price[strike]-1
df_calls['gain_stock']=(df_calls['strike']-ticker.price)/ticker.price

df_calls.stack(future_stack=True).head(40)
# df['stockgain']=(df['strike']-ticker.price)/ticker.price

contractSymbol                        
ADBE251024C00250000  strike               2.50e+02
                     lastPrice            1.00e+02
                     bid                  8.40e+01
                     ask                  9.19e+01
                     change               0.00e+00
                     volume                    NaN
                     openInterest         1.00e+00
                     impliedVolatility    2.17e+00
                     inTheMoney           1.00e+00
                     gain_280             3.69e+00
                     gain_stock          -2.50e-01
ADBE251024C00280000  strike               2.80e+02
                     lastPrice            5.40e+01
                     bid                  0.00e+00
                     ask                  0.00e+00
                     change               0.00e+00
                     volume               2.00e+01
                     openInterest         0.00e+00
                     impliedVolatility    1.00e-05
                     inTheMoney           1.00e+00
                     gain_280             4.26e+00
                     gain_stock          -1.60e-01
ADBE251024C00285000  strike               2.85e+02
                     lastPrice            6.85e+01
                     bid                  5.15e+01
                     ask                  5.73e+01
                     change               0.00e+00
                     volume                    NaN
                     openInterest         1.50e+01
                     impliedVolatility    1.56e+00
                     inTheMoney           1.00e+00
                     gain_280             4.35e+00
                     gain_stock          -1.45e-01
ADBE251024C00290000  strike               2.90e+02
                     lastPrice            7.58e+01
                     bid                  3.96e+01
                     ask                  4.80e+01
                     change               0.00e+00
                     volume               4.00e+00
                     openInterest         4.00e+00
dtype: float64

In [87]:
df

,strike,expiry,call,put
0,230.0,2025-11-21,103.26,0.00
1,230.0,2025-11-28,103.26,0.00
2,230.0,2025-12-19,103.26,0.00
3,230.0,2026-01-16,103.26,0.00
4,230.0,2026-02-20,103.26,0.00
...,...,...,...,...
95,420.0,2025-11-21,0.00,86.74
96,420.0,2025-11-28,0.00,86.74
97,420.0,2025-12-19,0.00,86.74
98,420.0,2026-01-16,0.00,86.74


In [ ]:
q=opt_quotes(ticker.price)
# q
df_quote=ticker.option_chain().calls.query("strike.isin(@q)")

In [ ]:
def opt_quotes(price,quotes=20,step_size=10):
  start=int(price/step_size)*step_size-quotes/2*step_size
  # print(start)
  return [start+x*step_size for x in range(quotes)]

opt_quotes(ticker.price)

In [ ]:
range?

In [ ]:
math.round